In [1]:
import json
import os
import gzip
from nltk.tokenize import sent_tokenize
import re
import csv
from tqdm.notebook import tqdm

In [2]:
input_fn = '../data/heads/bighead_test.jsonl.gz'
output_fn = '../out/bighead/zero_that/test_extract.csv'
max_context_len = 1000

In [3]:
data_train = list(gzip.open(input_fn, 'rt'))

In [4]:
fh = open(output_fn, 'wt')
csv_writer = csv.writer(fh)

# write headers
csv_writer.writerow(["line", "sentence", "contains", "index"])

30

In [5]:
regex ="((I|you|You|he|He) (think|thinks)) (that)?[\w ]{12}"

In [6]:
for i, line in tqdm(enumerate(data_train), total=len(data_train)):
    tokenized_line = sent_tokenize(re.sub('\n', ' ', json.loads(line)["text"]))
    
    # maximize the context
    for j in range(len(tokenized_line)-1, -1, -1):
        result = re.search(regex, tokenized_line[j])
        
        if result != None:
            index = result.span()[1]
            
            has_that = result.group(4) == "that"
            offset = 17
            index = result.span()[1] - offset
            
            if not has_that:
                index += 4
            
            context = ' '.join(tokenized_line[:j])
            context = context[-max_context_len:]  
            line_with_context = context + ' ' + tokenized_line[j]
            
            csv_writer.writerow([i, line_with_context, has_that, len(context) + 1 + index])
        
            # to go to next pile example
            break

  0%|          | 0/214584 [00:00<?, ?it/s]

In [7]:
fh.close()

In [8]:
i = len(context) + index
line_with_context[:i], line_with_context[i:]

('en, we decided to take a look at what chat programs we could use that would respect our privacy. Unfortunately, at the time, all other existing implementations were either too convoluted to convince our friends to use or were proprietary, so we decided that Tox was a necessary project. How are people using it? Right now, people are using Tox to talk with their family in a more secure way than what other big-name, proprietary competitors offer. People from all over the world are joining group chats to talk about their favorite hobbies, and friends are getting together to discuss weekend plans. We even have plugins that allow for Tox-to-IRC and vice versa conversations. Audio calling is available in a select few clients right now, so people are even using Tox to perhaps speak with one another while they play a video game. Tox itself is a protocol, so it can be adapted to anything you can imagine. Some people have even used Tox as a file sync, safely synchronizing between their computer